In [1]:
from faker import Faker
import pandas as pd
import random
import uuid
import hashlib
from datetime import datetime

In [45]:
fake = Faker("pt_BR")

# =========================================================
# CONFIGURAÇÕES
# =========================================================
TOTAL_REGISTROS = 100000
QTD_FORNECEDORES = 9000
QTD_LOTES = 800

SOURCE_SYSTEMS = ["ERP", "EXCEL", "LEGACY", "SALESFORCE"]
SOURCE_ENTITY = "transacoes_financeiras"

# =========================================================
# UTILITÁRIOS
# =========================================================
def gerar_hash(row):
    raw = "|".join([str(v) for v in row.values()])
    return hashlib.md5(raw.encode()).hexdigest()

def escolha_ponderada(valores, pesos):
    return random.choices(valores, weights=pesos, k=1)[0]

# =========================================================
# DATAS
# =========================================================
def gerar_data_transacao():
    return escolha_ponderada(
        [
            fake.date_between("-3y", "today").strftime("%d/%m/%Y"),
            fake.date_between("-3y", "today").strftime("%d.%m.%Y"),
            fake.date_between("-3y", "today").strftime("%Y-%m-%d"),
            fake.date_between("-3y", "today").strftime("%d-%b-%Y"),
            fake.date_between("-3y", "today").strftime("%d/%m/%y"),
            "31/02/2025",
            None
        ],
        [20, 15, 20, 10, 10, 10, 15]
    )

def gerar_data_competencia():
    if random.random() < 0.15:
        return None

    ano = random.randint(2019, 2026)
    mes = random.randint(1, 12)

    meses = ["jan", "fev", "mar", "abr", "mai", "jun",
             "jul", "ago", "set", "out", "nov", "dez"]

    return random.choice([
        f"{mes:02d}/{ano}",
        f"{mes:02d}{ano}",
        f"{ano}-{mes:02d}",
        f"{meses[mes-1]}/{ano}",
        f"{ano}/T{random.randint(1,4)}",
        "Extra"
    ])

# =========================================================
# ID TRANSAÇÃO RAW (DEPENDE DO SOURCE)
# =========================================================
def gerar_id_transacao_raw(source_system, seq):
    if random.random() < 0.12:
        return None

    if source_system == "ERP":
        return str(1000 + seq)

    if source_system == "EXCEL":
        return str(5000 + seq)

    if source_system == "SALESFORCE":
        return f"OPP-{5000 + seq}"

    if source_system == "LEGACY":
        return f"TRX#{uuid.uuid4().hex[:6].upper()}"

    return f"INV-2026-{seq:03d}"

# =========================================================
# VALORES FINANCEIROS (COM ANOMALIAS)
# =========================================================
def formatar_valor_raw(valor_real, moeda):
    if random.random() <= 0.85:
        valor_base = int(round(valor_real))

        if moeda == "BRL":
            return random.choice([
                f"{valor_base}",
                f"{valor_base:,}".replace(",", "."),
                f"{valor_base},00",
                f"{valor_base:,}".replace(",", ".") + ",00",
                f"R${valor_base:,}".replace(",", ".") + ",00"
            ])
        else:
            return random.choice([
                f"{valor_base}",
                f"{valor_base}.00",
                f"{valor_base:,}",
                f"${valor_base:,}"
            ])

    return random.choice(["ABC", " ", None])

# =========================================================
# DESCRIÇÃO
# =========================================================
def gerar_descricao():
    padroes = [
        "Pagamento fornecedor",
        "Pgto fornecedor",
        "Pagamento ref NF",
        "Pgto ref NF",
        "Pagamento nota fiscal",
        "Pgto NF",
        "Ajuste financeiro",
        "Ajuste contábil",
        "Estorno pagamento"
    ]

    return escolha_ponderada(
        [
            random.choice(padroes),
            f"{random.choice(padroes)} NF {random.randint(10000,99999)}",
            random.choice(padroes)[:random.randint(8, 18)],
            None,
            ""
        ],
        [40, 25, 15, 10, 10]
    )

# =========================================================
# FORMA / STATUS
# =========================================================
def gerar_forma_pagamento():
    return random.choice([
        "Cartão de Crédito",
        "Cartão Crédito",
        "CC",
        "Boleto",
        "BOLETO",
        "PIX",
        "Transferência PIX",
        "Dinheiro",
        "Espécie",
        None
    ])

def gerar_status_pagamento():
    return escolha_ponderada(
        ["Pago", "Pendente", "Cancelado", "Estornado", None],
        [45, 20, 10, 10, 15]
    )

# =========================================================
# FORNECEDORES (ID ÚNICO + MOEDA FIXA)
# =========================================================
fornecedores = []

for i in range(QTD_FORNECEDORES):
    id_fornecedor_raw = random.choice([
        f"FORN-{i:06d}",
        f"ACC-{i:06d}",
        f"{1000000 + i}"
    ])

    moeda = escolha_ponderada(["BRL", "USD"], [80, 20])

    fornecedores.append({
        "id_fornecedor_raw": id_fornecedor_raw,
        "media_pagamento": random.uniform(800, 7000),
        "desvio": random.uniform(0.05, 0.15),
        "moeda": moeda
    })

# =========================================================
# GERAÇÃO DOS DADOS
# =========================================================
dados = []
row_seq = 1
registros_por_lote = TOTAL_REGISTROS // QTD_LOTES

for _ in range(QTD_LOTES):
    ingestion_id = str(uuid.uuid4())
    source_system = random.choice(SOURCE_SYSTEMS)

    for _ in range(registros_por_lote):
        fornecedor = random.choice(fornecedores)

        eh_anomalia = random.random() < 0.06

        if eh_anomalia:
            fator = random.choice([0.01, 0.03, 0.05, 8, 12, 20, 50])
            valor_real = fornecedor["media_pagamento"] * fator
        else:
            valor_real = fornecedor["media_pagamento"] * random.uniform(
                1 - fornecedor["desvio"],
                1 + fornecedor["desvio"]
            )

        row = {
            "id_transacao_raw": gerar_id_transacao_raw(source_system, row_seq),
            "data_transacao": gerar_data_transacao(),
            "data_competencia": gerar_data_competencia(),
            "id_area_raw": escolha_ponderada(["ADM", "FIN", "TI", "0", None], [30, 30, 15, 10, 15]),
            "id_fornecedor_raw": fornecedor["id_fornecedor_raw"],
            "tipo_transacao": escolha_ponderada(["Receita", "Despesa", "Ajuste", None], [40, 40, 10, 10]),
            "valor_bruto": formatar_valor_raw(valor_real, fornecedor["moeda"]),
            "valor_liquido": formatar_valor_raw(valor_real * random.uniform(0.8, 0.97), fornecedor["moeda"]),
            "moeda": fornecedor["moeda"],
            "descricao": gerar_descricao(),
            "id_categoria_raw": escolha_ponderada(["DESP", "RECEITA", "100", None], [35, 25, 20, 20]),
            "forma_pagamento": gerar_forma_pagamento(),
            "status_pagamento": gerar_status_pagamento(),
            "ingestion_id": ingestion_id,
            "ingestion_ts": datetime.now(),
            "source_system": source_system,
            "source_entity": SOURCE_ENTITY,
            "row_seq": row_seq
        }

        row["raw_row_hash"] = gerar_hash(row)
        dados.append(row)
        row_seq += 1

df_raw = pd.DataFrame(dados)


In [ ]:
df_raw.head(10)

In [ ]:
df_raw[df_raw['id_fornecedor_raw'] == 'ACC-002902']

In [58]:
df_raw.to_csv("raw_transacoes_financeiras.csv", index=False, sep=";")